# Compute Energies of Random Offsets
Form a training set for approximate hessians by computing energies at many random displacements.

In [ ]:
from jitterbug.utils import make_calculator
from ase.io import write, read
from ase.db import connect
from ase import Atoms
from pathlib import Path
from tqdm import tqdm 
import numpy as np
import os

Configuration

In [ ]:
starting_geometry = '../data/exact/water_b3lyp_def2-svpd.xyz'
threads = min(os.cpu_count(), 12)
step_size: float = 0.005 # Perturbation amount, used as maximum L2 norm

Derived

In [ ]:
run_name = Path(starting_geometry).name[:-4]
name, method, basis = run_name.split("_")

In [ ]:
if not Path(starting_geometry).exists():
    raise ValueError('Cannot find file')

## Load in the Relaxed Structure
We generated a relaxed structure in the previous notebook

In [ ]:
atoms = read(starting_geometry)
atoms

## Compute many random energies
Compute $3N + 3N(3N+1)/2 + 1$ energies with displacements sampled [on the unit sphere](https://mathoverflow.net/questions/24688/efficiently-sampling-points-uniformly-from-the-surface-of-an-n-sphere). This is enough to fit the Jacobian and Hessian exactly plus a little more

Prepare the output directory

In [ ]:
out_dir = Path('data') / 'random-dir-same-dist'
out_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
db_path = out_dir / f'{run_name}_d={step_size:.2e}.db'

Add the relaxed geometry if needed

In [ ]:
if not db_path.is_file():
    with connect(db_path) as db:
        db.write(atoms)

Make the calculator

In [ ]:
calc = make_calculator(method, basis, threads=threads)

Generate the energies

In [ ]:
n_atoms = len(atoms)
to_compute = 3 * n_atoms + 3 * n_atoms * (3 * n_atoms + 1) // 2 + 1
print(f'Need to run {to_compute} calculations for full accuracy.')

In [ ]:
with connect(db_path) as db:
    done = len(db)
print(f'Already done {done}. {to_compute - done} left to do.')

In [ ]:
pbar = tqdm(total=to_compute)
pbar.update(done)
for i in range(to_compute - done):
    # Sample a perturbation
    disp = np.random.normal(0, 1, size=(n_atoms * 3))
    disp /= np.linalg.norm(disp)
    disp *= step_size * len(atoms) 
    disp = disp.reshape((-1, 3))
    
    # Subtract off any translation
    disp -= disp.mean(axis=0)[None, :]

    # Make the new atoms
    new_atoms = atoms.copy()
    new_atoms.positions += disp

    # Compute the energy and store in the db
    new_atoms.calc = calc
    new_atoms.get_potential_energy()
    with connect(db_path) as db:
        db.write(new_atoms)

    pbar.update(1)